>### 🚩 *Create a free WhyLabs account to get more value out of whylogs!*<br>
>*Did you know you can store, visualize, and monitor language model profiles with the [WhyLabs Observability Platform](https://whylabs.ai/whylogs-free-signup?utm_source=github&utm_medium=referral&utm_campaign=langkit_behavior_monitoring)? Sign up for a [free WhyLabs account](https://whylabs.ai/whylogs-free-signup?utm_source=github&utm_medium=referral&utm_campaign=langkit_behavior_monitoring) to leverage the power of LangKit and WhyLabs together!*

# Customizing LangKit Configuration

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/whylabs/langkit/blob/main/langkit/examples/LangKit_Config.ipynb)


In [ ]:
%pip install langkit[all]

LangKit has a few global variables that allow you to customize the parameters of some of its UDFs. Import and adjust these values **before** importing other LangKit modules. `langkit.prompt_column` and `langkit.response_column` are the names of the promt and response columns. UDFs are registered to apply to these columns, so it's best not to change them except in advanced use cases.
`langkit.lang_config` is an instance of the `LangKitConfig` class (see below), and serves as the default source of UDF parameters. Some LangKit modules allow you pass a custom `LangKitConfig` instance into their `init()` function, but you can also adjust the values in the default instance. Modules may or may not obey changes to `lang_config` between `init()` calls, but it's safe to call `init()` multiple times.


```
class LangKitConfig:
    pattern_file_path: str = field(
        default_factory=lambda: _resource_filename("pattern_groups.json")
    )
    theme_file_path: str = field(
        default_factory=lambda: _resource_filename("themes.json")
    )
    transformer_name: str = "sentence-transformers/all-MiniLM-L6-v2"
    topics: list = field(
        default_factory=lambda: [
            "law",
            "finance",
            "medical",
            "education",
            "politics",
            "support",
        ]
    )
    nlp_scores: list = field(
        default_factory=lambda: [
            "bleu",
            "rouge",
            "meteor",
        ]
    )
    reference_corpus: str = ""
```



As an example, let's disable all the NLP scores by setting `langkit.lang_config.nlp_scores` to an empty list before importing the `nlp_scores` module. We can also set the reference corpus. When we log, no extra columns are produced because no NLP score UDFs run.

In [2]:
from langkit import lang_config, response_column

lang_config.nlp_scores = []
lang_config.reference_corpus = "The quick brown fox jumps over the lazy dog"

from langkit import nlp_scores as ns
from whylogs.experimental.core.udf_schema import udf_schema
import whylogs as why

data = {response_column: "The quick dog jumps over the lazy brown fox"}

view = why.log(data, schema=udf_schema()).profile().view()
print(view.get_columns().keys())

dict_keys(['response'])


Now let's re-initialize the module to activate the bleu score.

In [3]:
ns.init(scores=["bleu"])
view = why.log(data, schema=udf_schema()).profile().view()

print(view.get_columns().keys())

dict_keys(['response', 'response.bleu_score'])


Note that you can add scores, but you can't retract ones that are already registered:

In [4]:
ns.init(scores=[])
view = why.log(data, schema=udf_schema()).profile().view()

print(view.get_columns().keys())

dict_keys(['response', 'response.bleu_score'])
